In [ ]:
#Step (1): Read Data
import pandas as pd #Read data
import numpy as np  #Array

# Read in dataset (encoding = 'Big5'可顯示出繁體中文)
data_train = pd.read_excel("01train_fix.xlsx")
data_test = pd.read_excel("01test.xlsx")
data_out = pd.read_excel("01test.xlsx")
data_train.head() # Show data (看看是否讀取成功)

In [ ]:
#drop 上網試試看,心肌、平均app不drop
columns_to_drop = ['外型平均曲率', '手機外殼/包膜指標', '手機螢幕溫度(最大)', 'CPU效能等級', '平均APP運作指標/小時', '新機/二手/老舊']
data_train.drop(columns=columns_to_drop, axis=1, inplace=True)
data_test.drop(['id'] + columns_to_drop, axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler


columns_to_normalize = [ '手機螢幕溫度(平均)', '手機背蓋溫度(平均)','手機電池電量', 'CPU跑分階級','內容容量比率','充電時間指標', '通話頻率指標', '上網頻率指標']

# 初始化標準化物件
min_max_scaler = MinMaxScaler()

# 將資料標準化
data_train[columns_to_normalize] = min_max_scaler.fit_transform(data_train[columns_to_normalize])
data_test[columns_to_normalize] = min_max_scaler.transform(data_test[columns_to_normalize])

# 顯示標準化後的資料
print(data_train.head())

In [ ]:
data_train.describe()

In [ ]:
data_test.describe()

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler  #建構standardization的物件

#name_list_outlier = ['手機螢幕溫度(平均)','手機螢幕溫度(最大)','手機背蓋溫度(平均)','平均APP運作指標/小時','充電時間指標',
                #'手機電池電量','手機外殼/包膜指標','外型平均曲率','上網頻率指標',
                #'通話頻率指標','新機/二手/老舊','內容容量比率','CPU效能等級','CPU跑分階級']

#name_list_outlier = ['手機螢幕溫度(最大)' ,'手機背蓋溫度(平均)','充電時間指標','手機電池電量','通話頻率指標',
                        #'新機/二手/老舊','內容容量比率','CPU效能等級','CPU跑分階級']

name_list_outlier =[ '手機背蓋溫度(平均)',  '內容容量比率']
name_pre_outlier = ['Mobile Screen Temperature (Max)','Mobile Phone Back Cover Temperature (Average)','Average APP Operation Index/Hour','Mobile Phone Battery Level','Content Capacity Ratio']


#離群值處理
def  outlier_del(data_frame,name_list_outlier,n1=5,n2=5):
    for i, name in enumerate(name_list_outlier):
        print(name)
        print("Shape Of The Before Ouliers: ", data_frame[name].shape)
    
        # 计算IQR
        Q1 = np.percentile(data_frame[name], 25)
        Q3 = np.percentile(data_frame[name], 75)
        IQR = Q3 - Q1

        # 定义离群值范围
        lower_bound = Q1 - n1 * IQR
        upper_bound = Q3 + n2 * IQR

        # 过滤离群值
        filtered_entries = ((data_frame[name] >= lower_bound) & (data_frame[name] <= upper_bound))
        data_frame = data_frame[filtered_entries]
    
        print("Shape Of The After Ouliers: ",data_frame[name].shape)
        print('===================================================================================================================')
    return data_frame

In [ ]:
class_distribution = data_train['Underclocking'].value_counts() #指定data中的label欄位，進行各類別數量的統計
print("Class Distribution:")
print(class_distribution)

In [ ]:
feature_ex =['手機螢幕溫度(平均)', '手機背蓋溫度(平均)', '充電時間指標', '手機電池電量', 'CPU跑分階級', '通話頻率指標', '內容容量比率', '上網頻率指標']
x_data = data_train[feature_ex]
y_data = data_train['Underclocking']

In [ ]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, criterion='gini',max_depth=6,random_state=42,n_jobs=-1)

# 定义 KFold 交叉验证对象
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 使用 cross_validate 进行交叉验证
cv_results = cross_validate(rf_model, x_data, y_data, cv=kfold, scoring='accuracy', return_train_score=True)

# 输出每个 fold 的训练得分和验证得分
for i in range(5):
    train_score = cv_results['train_score'][i]
    test_score = cv_results['test_score'][i]
    print(f"Fold {i+1} - Training Score: {train_score}, Validation Score: {test_score}")

# 输出平均训练得分和平均验证得分
mean_train_score = np.mean(cv_results['train_score'])
mean_test_score = np.mean(cv_results['test_score'])
print(f"\nMean Training Score: {mean_train_score}")
print(f"Mean Validation Score: {mean_test_score}")

In [ ]:
#Step (4): 把dataset分成訓練跟驗證
from sklearn.model_selection import train_test_split
#test_size=0.3 為比例-> training:test=7:3
trainX, testX, trainY, testY = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


rf_model = RandomForestClassifier(n_estimators=300, criterion='gini',max_depth=6,random_state=42,n_jobs=-1)

# 使用模型对训练集进行训练
rf_model.fit(trainX, trainY)

# 获取训练集和验证集的预测结果
Y_train_pred = rf_model.predict(trainX)
Y_test_pred = rf_model.predict(testX)

# 计算训练集和验证集的准确率
train_accuracy = accuracy_score(trainY, Y_train_pred)
test_accuracy = accuracy_score(testY, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# 打印分类报告和混淆矩阵
print("Classification Report:\n", classification_report(testY, Y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(testY, Y_test_pred))

In [ ]:
predicted_ans = rf_model.predict(data_test[feature_ex])
my_submission = pd.DataFrame({'id': data_out.id, 'Underclocking': predicted_ans})
my_submission.to_csv('submission.csv', index=False)
